https://github.com/4GeeksAcademy/Hector-random-forest.git

In [3]:
# Your code here
#Librerias
from numpy.random import seed, choice
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text, DecisionTreeRegressor
from collections import Counter
import numpy as np
from sklearn.metrics import *
from imblearn.metrics import specificity_score
import pandas as pd
import matplotlib.pyplot as plt
from pickle import dump

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/decision-tree-project-tutorial/main/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.shape

(768, 9)

In [6]:
df = df.drop_duplicates().reset_index(drop = True)
df.shape

(768, 9)

In [7]:
#Haremos la seleccion de los datos de entenaminenot y test
#En este caso no es necesario normalizar las variables
X = df.drop("Outcome", axis = 1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
60,2,84,0,0,0,0.0,0.304,21
618,9,112,82,24,0,28.2,1.282,50
346,1,139,46,19,83,28.7,0.654,22
294,0,161,50,0,0,21.9,0.254,65
231,6,134,80,37,370,46.2,0.238,46


In [8]:
X_test.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
668,6,98,58,33,190,34.0,0.430,43
324,2,112,75,32,0,35.7,0.148,21
624,2,108,64,0,0,30.8,0.158,21
690,8,107,80,0,0,24.6,0.856,34
473,7,136,90,0,0,29.9,0.210,50


In [9]:
#Lo convertiremos a CSV
'''
X_train_csv=X_train
X_test_csv=X_test
X_train_csv["Outcome"] = y_train.values
X_test_csv["Outcome"] = y_test.values
X_train_csv.to_csv("../data/processed/clean_train.csv", index = False)
X_test_csv.to_csv("../data/processed/clean_test.csv", index = False)
'''

'\nX_train_csv=X_train\nX_test_csv=X_test\nX_train_csv["Outcome"] = y_train.values\nX_test_csv["Outcome"] = y_test.values\nX_train_csv.to_csv("../data/processed/clean_train.csv", index = False)\nX_test_csv.to_csv("../data/processed/clean_test.csv", index = False)\n'

In [10]:
#Creamos una clase para personalizar la creacion de random forest
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from numpy.random import seed, choice
from collections import Counter

class RandomForestCustom:

    def __init__(self, n_estimators, random_state, max_depth, min_samples_leaf, max_features, X, y):
        self.n_estimators = n_estimators
        self.random_state = random_state
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.X = X
        self.y = y
        self.models = []

    def get_bootstrap_dataset(self):
        seed(self.random_state)
        ls_idx_features, ls_data = [], []
        for i in range(self.n_estimators):
            idx_rows_tree = choice(len(self.X), len(self.X), replace=True)
            idx_cols_tree = choice(self.X.shape[1], self.max_features, replace=False)
            ls_idx_features.append(idx_cols_tree)
            if isinstance(self.X, np.ndarray):
                ls_data.append((self.X[idx_rows_tree, :][:, idx_cols_tree], self.y[idx_rows_tree]))
            else:
                ls_data.append((self.X.iloc[idx_rows_tree, :].iloc[:, idx_cols_tree], self.y.iloc[idx_rows_tree]))
        return ls_idx_features, ls_data

    def fit(self):
        ls_idx_features, ls_data = self.get_bootstrap_dataset()
        for i, data in enumerate(ls_data):
            X, y = data
            tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf, max_features=self.max_features).fit(X, y)
            self.models.append((ls_idx_features[i], tree))

    def predict(self, X):
        ls_predictions = []
        for idxs, tree in self.models:
            if isinstance(X, np.ndarray):
                X_to_predict = X[:, idxs]
            else:
                X_to_predict = X.iloc[:, idxs]
            yhats = tree.predict(X_to_predict)
            ls_predictions.append(yhats)
        predictions = np.vstack(ls_predictions)
        final_predictions = [Counter(predictions[:, i]).most_common(1)[0][0] for i in range(predictions.shape[1])]
        return final_predictions


In [11]:
rf_simple = RandomForestCustom(n_estimators=5, random_state=42, max_depth=2, min_samples_leaf=20, max_features=X_train.shape[1]//2, X=X_train, y=y_train)
rf_simple.fit()
rf_complex = RandomForestCustom(n_estimators=100, random_state=42, max_depth=20, min_samples_leaf=1, max_features=int(X_train.shape[1]*0.8), X=X_train, y=y_train)
rf_complex.fit()

In [12]:
y_pred_train_simple = rf_simple.predict(X_train)
y_pred_test_simple = rf_simple.predict(X_test)

y_pred_train_complex = rf_complex.predict(X_train)
y_pred_test_complex = rf_complex.predict(X_test)

In [13]:
def get_metrics(y_train, y_test, y_pred_train, y_pred_test):
    # Calcular métricas para el conjunto de entrenamiento
    train_accuracy = accuracy_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train)
    train_auc = roc_auc_score(y_train, y_pred_train)
    train_precision = precision_score(y_train, y_pred_train)
    train_recall = recall_score(y_train, y_pred_train)
    train_specificity = specificity_score(y_train, y_pred_train)

    # Calcular métricas para el conjunto de prueba
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_specificity = specificity_score(y_test, y_pred_test)

    # Calcular la diferencia entre métricas de entrenamiento y prueba
    diff_accuracy = train_accuracy - test_accuracy
    diff_f1 = train_f1 - test_f1
    diff_auc = train_auc - test_auc
    diff_precision = train_precision - test_precision
    diff_recall = train_recall - test_recall
    diff_specificity = train_specificity - test_specificity

    # Crear un DataFrame con los resultados
    metrics_df = pd.DataFrame([[train_accuracy, train_f1, train_auc, train_precision, train_recall, train_specificity],[test_accuracy, test_f1, test_auc, test_precision, test_recall, test_specificity],[diff_accuracy, diff_f1, diff_auc, diff_precision, diff_recall, diff_specificity]],
                              columns = ['Accuracy', 'F1', 'AUC', 'Precision', 'Recall', 'Specificity'],
                              index = ['Train','Test', 'Diferencia'])

    return metrics_df

In [14]:
get_metrics(y_train, y_test, y_pred_train_complex, y_pred_test_complex)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000
Test,0.707792,0.60177,0.687879,0.586207,0.618182,0.757576
Diferencia,0.292208,0.39823,0.312121,0.413793,0.381818,0.242424


In [15]:
get_metrics(y_train, y_test, y_pred_train_simple, y_pred_test_simple)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.741042,0.627635,0.714803,0.626168,0.629108,0.800499
Test,0.681818,0.573913,0.663636,0.550000,0.600000,0.727273
Diferencia,0.059224,0.053722,0.051167,0.076168,0.029108,0.073226


In [16]:
#Analizacermos las predicciones
y_pred = rf_simple.predict(X_test)
y_pred

[np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.in

In [17]:
accuracy_score(y_test, y_pred)

0.6818181818181818

In [18]:
#Guradadod del modelo
'''
dump(rf_simple, open("../models/tree_classifier_crit-entro_maxdepth-5_minleaf-4_minsplit2_42.sav", "wb"))
'''

'\ndump(rf_simple, open("../models/tree_classifier_crit-entro_maxdepth-5_minleaf-4_minsplit2_42.sav", "wb"))\n'